<a href="https://colab.research.google.com/github/bchandhu/FakeCurrencyDetectionSystem/blob/master/Fake_currency_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!git clone https://github.com/bchandhu/FakeCurrencyDetectionSystem.git


fatal: destination path 'FakeCurrencyDetectionSystem' already exists and is not an empty directory.


In [10]:
# Import all necessary modules
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt


In [11]:
#define height and width of the image
height=300
width=300

#create a ResNet50 model instance without the top layer as we will add our own top layer
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

In [12]:
#define directory containing training and validation data
train_dir="/content/FakeCurrencyDetectionSystem/Dataset/Training"
validation_dir="/content/FakeCurrencyDetectionSystem/Dataset/Validation"

#number of batches the data has to be divided into
batch_size=8

#create datagen and generator to load the data from training directory
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)

#create datagen and generator to load the data from validation directory
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(validation_dir,target_size=(height,width),batch_size=batch_size)

Found 35 images belonging to 2 classes.
Found 7 images belonging to 2 classes.


In [13]:
#our own model which will be added onto the ResNet50 model
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False

    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)

    predictions=Dense(num_classes,activation='softmax')(x)

    finetune_model=Model(inputs=base_model.input,outputs=predictions)

    return finetune_model

class_list=['Real','Fake'] #the labels of our data
FC_Layers=[1024,1024]
dropout=0.5

finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=FC_Layers,num_classes=len(class_list))

In [14]:
#define number of epochs(the number of times the model will be trained) and number of training images
num_epochs=100
num_train_images=35

In [15]:
#checkpoint in case anything goes wrong
checkpoint=ModelCheckpoint("Final_model.h5",monitor='val_acc',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',period=1)
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=40,verbose=1,mode="auto")

#compile the model before using
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizers.SGD(lr=0.000001,momentum=0.9),metrics=['accuracy'])

#train the model
finetune_model.fit_generator(generator=train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])

#save the model
finetune_model.save_weights("Final_model.h5")

<ipython-input-15-c4ac6982538f>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  finetune_model.fit_generator(generator=train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])


Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 1138.2755 - accuracy: 0.4815

4/4 [==============================] - 28s 6s/step - loss: 1138.2755 - accuracy: 0.4815 - val_loss: 10736.8018 - val_accuracy: 0.2857
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 16614599.0000 - accuracy: 0.6667

4/4 [==============================] - 19s 5s/step - loss: 16614599.0000 - accuracy: 0.6667 - val_loss: 183455465472.0000 - val_accuracy: 0.2857
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 30128206127899753309306374586368.0000 - accuracy: 0.3704

4/4 [==============================] - 22s 5s/step - loss: 30128206127899753309306374586368.0000 - accuracy: 0.3704 - val_loss: nan - val_accuracy: 0.7143
Epoch 4/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4688

...

4/4 [==============================] - 21s 5s/step - loss: nan - accuracy: 0.4815 - val_loss: nan - val_accuracy: 0.2857
Epoch 97/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.3704

4/4 [==============================] - 21s 5s/step - loss: nan - accuracy: 0.3704 - val_loss: nan - val_accuracy: 0.2857
Epoch 98/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.3333

4/4 [==============================] - 18s 5s/step - loss: nan - accuracy: 0.3333 - val_loss: nan - val_accuracy: 0.2857
Epoch 99/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4444

4/4 [==============================] - 18s 5s/step - loss: nan - accuracy: 0.4444 - val_loss: nan - val_accuracy: 0.2857
Epoch 100/100
4/4 [==============================] - ETA: 0s - loss: nan - accuracy: 0.3704

4/4 [==============================] - 27s 7s/step - loss: nan - accuracy: 0.3704 - val_loss: nan - val_accuracy: 0.2857


In [29]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'finetune_model' is already defined and loaded with weights
finetune_model.load_weights("Final_model.h5")

def test_fake_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    img = np.asarray(img)
    plt.imshow(img)
    img = np.expand_dims(img, axis=0)
    output = finetune_model.predict(img)
    if output[0][0] > output[0][1]:
        print("Prediction: Real")
    else:
        print("Prediction: Fake")

def test_real_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    img = np.asarray(img)
    plt.imshow(img)
    img = np.expand_dims(img, axis=0)
    output = finetune_model.predict(img)
    if output[0][0] > output[0][1]:
        print("Prediction: Fake")
    else:
        print("Prediction: Real")

# Testing fake images
test_fake_image("/content/FakeCurrencyDetectionSystem/Dataset/Testing/Fake.jpeg")

# Testing real images
test_real_image("/content/FakeCurrencyDetectionSystem/Dataset/Testing/Real.jpg")




FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Final_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Final_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)